### Applico un algoritmo di rete neurale densa (Dense Neural Network) a un dataset di classificazione

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

sns.set_theme()

random_state = 42

In [ ]:
df=pd.read_csv('/content/dataset_titanic.csv')
df.isna().sum()

In [ ]:
def preprocess_titanic_dataset(df_train, df_test):
    df_train = df_train.copy(deep=True)
    df_test = df_test.copy(deep=True)

    df_X_train_preprocessed = pd.DataFrame(index=df_train.index)
    df_X_test_preprocessed = pd.DataFrame(index=df_test.index)

    series_y_train = pd.Series(index=df_train.index, data=df_train['Survived'])
    series_y_test = pd.Series(index=df_test.index, data=df_test['Survived'])

    # Fill nan values for column 'Embarked'
    embarked_mode = df_train['Embarked'].mode()[0]
    df_train['Embarked'] = df_train['Embarked'].fillna(embarked_mode)
    df_test['Embarked'] = df_test['Embarked'].fillna(embarked_mode)

    # Fill nan values for column 'Age'
    age_median = df_train['Age'].median()
    df_train['Age'] = df_train['Age'].fillna(age_median)
    df_test['Age'] = df_test['Age'].fillna(age_median)

    # Standardization for numerical features
    features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
    ss = StandardScaler()
    X_train_std = ss.fit_transform(df_train[features])
    X_test_std = ss.transform(df_test[features])
    df_X_train_preprocessed[features] = X_train_std
    df_X_test_preprocessed[features] = X_test_std

    # One-hot-encoding for categorical features
    features = ['Sex', 'Embarked']
    ohe = OneHotEncoder(drop='first', sparse_output=False,handle_unknown='ignore')
    X_train_ohe = ohe.fit_transform(df_train[features])
    X_test_ohe = ohe.transform(df_test[features])
    features_names_out = ohe.get_feature_names_out()
    df_X_train_preprocessed[features_names_out] = X_train_ohe
    df_X_test_preprocessed[features_names_out] = X_test_ohe

    return df_X_train_preprocessed, df_X_test_preprocessed, series_y_train, series_y_test


def evaluate_model(model, X_train, y_train, X_test, y_test, show_confusion_matrix=False):
  train_probs = model.predict(X_train).ravel()
  test_probs = model.predict(X_test).ravel()

  y_train_pred = (train_probs > 0.5).astype(int)
  y_test_pred = (test_probs > 0.5).astype(int)

  acc_train = accuracy_score(y_train, y_train_pred)
  acc_test = accuracy_score(y_test, y_test_pred)

  print(f"accuracy train: {acc_train:.3f}")
  print(f"accuracy test: {acc_test:.3f}")

  cf_matrix_test = confusion_matrix(y_test, y_test_pred)

  if show_confusion_matrix:
    ax = sns.heatmap(cf_matrix_test, annot=True, fmt='d')
    ax.set_title('Confusion matrix on test set')
    ax.set_xlabel('Predicted values')
    ax.set_ylabel('True values')
    plt.show()

  return acc_train, acc_test, cf_matrix_test

In [ ]:
df_train, df_test = train_test_split(df, test_size=.2, random_state=random_state)

df_X_train_preprocessed, df_X_test_preprocessed, series_y_train, series_y_test = preprocess_titanic_dataset(df_train, df_test)

X_train = df_X_train_preprocessed.values

X_test = df_X_test_preprocessed.values

y_train = series_y_train.values

y_test = series_y_test.values

In [ ]:
# Initialize the model
model = keras.Sequential()

# Add a Dense (fully connected) layer with 64 units(neurons) and ReLU activation
model.add(layers.Dense(64, activation='relu', input_shape=(df_X_train_preprocessed.shape[1],)))

# Add another layer
model.add(layers.Dense(64, activation='relu'))

# Add a final output layer with 1 unit for binary classification
model.add(layers.Dense(1, activation='sigmoid'))

# Display the model summary
model.summary()

In [ ]:
# Compile the model by specifying the optimizer, loss function, and metrics to
# track
model.compile(
    optimizer='adam',               # Adam optimizer for adaptive learning rate adjustments
    loss='binary_crossentropy',
    metrics=['accuracy']            # Metric to track: accuracy of predictions
)

In [ ]:
n_epochs = 25

# Train the model on the training data with specified parameters
history = model.fit(
    X_train,                # Training input data
    y_train,                # Training labels (target values)
    epochs=n_epochs,               # Number of times the entire dataset will be iterated over
    batch_size=64,          # Number of samples per gradient update
    validation_split=0.2    # Fraction of training data (20%) to use as validation data
)

In [ ]:
df_training_history = pd.DataFrame(history.history, index=range(1, n_epochs+1))

df_training_history.index.name = 'epoch'

df_training_history

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 7))

sns.lineplot(x=df_training_history.index, y=df_training_history['loss'], ax=axs[0], label='Loss training')

sns.lineplot(x=df_training_history.index, y=df_training_history['val_loss'], ax=axs[0], label='Loss validation')

sns.lineplot(x=df_training_history.index, y=df_training_history['accuracy'], ax=axs[1], label='Accuracy training')

sns.lineplot(x=df_training_history.index, y=df_training_history['val_accuracy'], ax=axs[1], label='Accuracy validation')

plt.tight_layout()

plt.show()

In [ ]:
evaluate_model(model, X_train, y_train, X_test, y_test, show_confusion_matrix=True)